# medeiros_STORM_analysis

Notebook for STORM shape analysis in Medeiros, A. T., Gratz, S.J., Delgado, A., Ritt, J.T. and O’Connor-Giles, K. M. "Intersecting levels of molecular and organizational diversity combine to generate functional synaptic heterogeneity within and between excitatory neuronal subtypes".

Loads an Excel sheet containing fluorophore localizations grouped into clusters, and fits 2D [alpha shapes](https://doc.cgal.org/latest/Alpha_shapes_2/index.html) to them. These shapes defined an active zone area, allowing calculation of AZ size and channel density.

In [ ]:
# Example data

dbscan_file = '20220526_L1_NMJ1_A2R67_DBSCAN_v2.xlsx'
dbscan_sheet = 'B1_Ib_DBSCAN'
max_proj_file = 'MAX_20220526_CacHalo_JF646_L1_A2R67_000.nd2 - C=0.png'

## Initialize

In [ ]:
# Dependencies

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# CGAL package for point->shape analysis
from CGAL.CGAL_Kernel import Point_2
from CGAL.CGAL_Alpha_shape_2 import Alpha_shape_2, GENERAL, REGULAR

In [ ]:
from CGAL.CGAL_Kernel import area

In [ ]:
# The following line will enable interactive plots within the notebook, and
# requires ipympl to be installed. You can comment it out to use default
# plotting, or try "%matplotlib qt" to pop out interactive plots in new
# windows.

%matplotlib widget

In [ ]:
# Utility functions

def um_to_px(coords, pxum=6.25):
    '''
    Convert coordinates from micrometers in the STORM file to 
    pixels in the associated confocal image.
    
    pxum is pixels per micron, found in the scope metadata.
    '''
    # Assumes Matplotlib convention for pixel locations
    return coords*pxum - 0.5 

def make_alpha_shape(coords, alpha):
    '''
    Return the CGAL object for alpha shape from the point cloud.
    Note the object contains shape information for all choices of
    alpha, but we set a specific value after creation, to be used
    in all subsequent analyses.
    
    coords should be an N by 2 array of (x,y) locations in micrometers.
    
    alpha is a non-negative float that determines shape complexity; larger
    means simpler shape.
    If zero, returns the original set of points as a set of distinct shapes.
    At a large enough alpha, the shape becomes the convex hull, and then no
    further changes occur with larger alpha.
    Note that the rest of the code in this notebook assumes only a single
    shape is returned for each cluster, i.e. that alpha is not too small.
    
    In the CGAL implementation, the "shape" is actually an object representing
    the entire family of shapes across alpha values (fixed by coords), but 
    a default value can be set.
    
    This function is just a convenience wrappper for CGAL.make_alpha_shape.
    '''
    
    # Point_2 is CGAL object for representing a point; cannot use
    # Numpy arrays directly.
    # SWIG wrapper for CGAL maps python lists onto C++ array pointers
    point_list = [Point_2(*cur_coord) for cur_coord in coords]
    
    cur_shape = Alpha_shape_2(point_list, alpha, GENERAL)
    cur_shape.make_alpha_shape(point_list)
    cur_shape.set_alpha(alpha) # Must be reset after fit to points

    return cur_shape

def get_az_area(alpha_shape):
    '''
    Find the active zone area, defined by a 2D alpha shape
    fit to a point cloud. 
    
    The input should be a CGAL Alpha_Shape_2 object, as returned
    from make_alpha_shape. Note that the code assumes a specific value
    of the alpha parameter has been set in the object; this ensures that
    all analyses with the object use the same alpha parameter.
    
    Approach is to take the polygonal area of the alpha_shape boundary.
    Does not require boundary to be convex, but it is assumed there is only a 
    single simple (non-intersecting) polygon given by the shape edges
    iterator.
    '''

    # Unfortunately, CGAL does not provide oriented boundary segments,
    # so we first have to link the vertices in order using the boundary
    # segment source/target labels

    # Aggregate all shape segments into array [x_from,y_from,x_to,y_to]
    edge_points = []
    for k in alpha_shape.alpha_shape_edges():
        p1 = alpha_shape.segment(k).source()
        p2 = alpha_shape.segment(k).target()
        edge_points.append( [p1.x(),p1.y(),p2.x(),p2.y()] )
    edge_points = np.array( edge_points )
    num_points = edge_points.shape[0]

    # Now find an ordering by linking "from/to" points across segments
    # These appear to be copies of the same values, so exact equality
    # tests work; it might be more robust to allow for floating point error.    
    rows_to_search = np.arange(num_points).tolist()
    shape_vertices = np.full( (num_points,2), np.nan )
    next_point = edge_points[0,2:]
    for point_idx in range(num_points):
        next_test = np.all( edge_points[rows_to_search,:2] == next_point, axis=1 )
        next_idx = rows_to_search[ next_test.nonzero()[0][0] ]

        shape_vertices[point_idx] = edge_points[next_idx,:2]
        next_point = edge_points[next_idx,2:]
        rows_to_search.remove(next_idx)  # Already linked point, do not search again
    
    # Now use the "shoelace" formula to find area of oriented polygon
    x = shape_vertices[:,0]
    y = shape_vertices[:,1]
    A = 0.5*np.abs( np.dot(x[:-1],y[1:]) + x[-1]*y[0]
                     - x[0]*y[-1] - np.dot(x[1:],y[:-1]) )
    
    return A

def plot_img_coords_alpha(Im, cluster_shapes, alpha_param=None):
    '''
    Plot segmented alpha_shapes and localizations on top of confocal image.
    
    Im is an RGB array.
    
    cluster_shapes is a list of CGAL alpha_shapes.
    
    alpha_param (non-negative float) is optional to change the alpha parameter;
    best practice is to set in the Alpha_Shape_2 object during creation for
    consistency across all analyses.
    
    Requires um_to_px utility function to align the two scope coordinates.
    '''
    plt.figure()
    plt.imshow(Im[:,:,0], cmap = "gray")

    for cur_shape in cluster_shapes:
        if alpha_param is not None:
            # Setting here may have side effects !
            cur_shape.set_alpha(alpha_param)
        cluster_color = np.random.rand(1,3)
        
        # Draw the edges of the current shape
        for cur_edge in cur_shape.alpha_shape_edges():
            cur_segment = cur_shape.segment(cur_edge)

            X_from = cur_segment.source().x()
            X_to = cur_segment.target().x()
            Y_from = cur_segment.source().y()
            Y_to = cur_segment.target().y()        
            X_px = um_to_px( np.array([X_from,X_to]) )
            Y_px = um_to_px( np.array([Y_from,Y_to]) )
            
            plt.plot(X_px,Y_px,color=cluster_color)
        
        # Draw the individual points of the current shape
        coords = um_to_px( np.array( [(c.x(),c.y()) for c in cur_shape.points()] ))
        plt.scatter(coords[:,0], coords[:,1], 5,color=cluster_color)

    plt.axis('image')

In [ ]:
# Primary analysis function

def get_cluster_stats(dbscan_file, dbscan_sheet, alpha_param):
    '''
    Analyzes distribution of fluorophores in active zones using alpha shape geometry.
    
    dbscan_file, dbscan_sheet are strings giving the Excel file and sheet to load.
    
    alpha_param is a non-negative float used as complexity parameter (bigger is simpler).
    '''
    
    # TODO: Remove hard coded columns
    data = pd.read_excel(dbscan_file, sheet_name=dbscan_sheet, usecols=[1, 2, 5, 37])
    
    # Remove lateral localization accuracy points below 50nm (0.05 microns)
    print(f'Number of poorly localized points: {np.sum( data["Lateral Localization Accuracy"] > 0.05)}')
    data = data.loc[data["Lateral Localization Accuracy"] <= 0.05]           
    
    # Get list of cluster IDs, and reject noise cluster (ID==-1) if present
    cluster_ids = data.iloc[:,3].unique()
    cluster_ids = cluster_ids[cluster_ids>=0]

    cluster_stats = np.full( (len(cluster_ids),5), np.nan )
    cluster_shapes = []
    for kk, cid in enumerate(cluster_ids):
        print(f'Working on cluster {cid}')
        coords = data[data['Clusters - ID']==cid].iloc[:,0:2].to_numpy()
        num_coords = len(coords)
        
        cur_shape = make_alpha_shape(coords, alpha_param)
        cluster_shapes.append( cur_shape )
        
        # Compute various stats
        cluster_stats[kk,0] = cid  # Identifier
        cluster_stats[kk,1] = get_az_area(cur_shape)  # Area, um^2
        cluster_stats[kk,2] = 2*np.sqrt(cluster_stats[kk,1]/np.pi)  # Equivalent diameter, um^2
        cluster_stats[kk,3] = num_coords / cluster_stats[kk,1]  # Density, num/um^2
        cluster_stats[kk,4] = num_coords   # Number of locs in cluster
     
    cluster_stats_pd = pd.DataFrame(cluster_stats,
                                    columns = ['Cluster_ID',
                                               'ClusterArea ( $\mu m^2$ )',
                                               'ClusterDiameter ( $\mu m$ )',
                                               'Density',
                                               '# of Locs'])
    
    return {"cluster_stats":cluster_stats_pd, "cluster_shapes":cluster_shapes}

## Load and analyze data

Expects an Excel workbook in the current directory, with a sheet containing STORM points.

Uses CGAL package for alpha shapes to find piecewise linear outlines of each active zone, and compute geometric properties like diameter and density.

As $\alpha$ (in the code as `alpha_param`) goes to infinity, the alpha shape becomes a convex hull. As $\alpha$ goes to zero, the active zone will be broken into distinct subclusters, eventually with each point its own cluster when $\alpha=0$. We aim for an $\alpha$ a little below the minimum that produces a convex hull. CGAL can find an optimal (minimal) $\alpha$ for each point cloud to be a single shape, but to avoid overfitting we choose a single larger value to apply to all clusters across all imaging sessions.

In [ ]:
alpha_param = 0.015  # Shape complexity

cluster_data = get_cluster_stats(dbscan_file, dbscan_sheet, alpha_param)

In [ ]:
cluster_data["cluster_stats"].sort_values(by = ['Cluster_ID'], ascending = True )

## Visualize

Plot the confocal max-projection image, the individual STORM points, and the alpha shape outlines.

In [ ]:
# Load max projection image from raw nd2 video wih Fiji/ImageJ to maintain alignment

Im = plt.imread(max_proj_file)

plot_img_coords_alpha(Im, cluster_data["cluster_shapes"]) 

#plt.savefig('Fig_alpha_shapes.svg', format='svg', dpi=300)